In [ ]:
from Bio import SeqIO
from Bio import AlignIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.Alphabet import IUPAC
import subprocess
import tempfile
import os
import re
from io import StringIO, BytesIO
from bokeh.io import output_notebook, push_notebook
from bokeh.layouts import column, gridplot
from bokeh.plotting import figure, show
from bokeh.models.tools import BoxZoomTool
from bokeh.models.tickers import FixedTicker
from bokeh.models import Range1d, LinearAxis, BoxAnnotation
from bokeh.models.callbacks import CustomJS
from bokeh.resources import INLINE, CDN
import numpy as np
from util import grouper

In [ ]:
output_notebook(resources=INLINE)

In [ ]:
ref_seq = "GCGAACAGTTCGGCTGGCGCGAGCCCCTGATGCTCTTCGTCCAGATCATCCTGATCGACAAGACCGGCTTCCATCCGAGTACGTGCTCGCTCGATGCGATGTTTCGCTTGGTGGTCGAATGGGCAGGTAGCCGGATCAAGCGTATGCAGCCGCCGCATTGCATCAGCCATGATGGATACTTTCTCGGCAGGAGCAAGGTGAGATGACAGGAGATCCTGCCCCGGCACTTCGCCCAATAGCAGCCAGTCCCTTCCCGCTTCAGTGACAACGTCGAGCACAGCTGCGCAAGGAACGCCCGTCGTGGCCAGCCACGATAGCCGCGCTGCCTCGTCCTGCAGTTCATTCAGGGCACCGGACAGGTCGGTCTTGACAAAAAGAACCGGGCGCCCCTGCGCTGACAGCCGGAACACGGCGGCATCAGAGCAGCCGATTGTCTGTTGTGCCCAGTCATAGCCGAATAGCCTCTCCACCCAAGCGGCCGGAGAACCTGCGTGCAATCCATCTTGTTCAATCATGCGAAACGATCCTCATCCTGTCTCTTGATCAGATCTTGATCCCCTGCGCCATCAGATCCTTGGCGGCAAGAAAGCCATCCAGTTTACTTTGCAGGGCTTCCCAACCTTACCAGAGGGCGCCCCAGCTGGCAATTCCGACGTCTAAGAAACCATTATTATCATGACATTAACCTATAAAAATAGTCCTCCGCTTATTGATATGCGTCTTCACCACAGCTAACACCACGTCGTCCCTATCTGCTGCCCTAGGTCTATGAGTGGTTGCTGGATAACTTTACGGGCATGCATAAGGCTCGTATAATATATTCAGGGAGACCACAACGGTTTCCCTCTACAAATAATTTTGTTTAACTTTAGTAAGGAGTCTAGACCATGAGTAAAGGCGAAGAATTGTTCACTGGCGTGGTACCGATCCTGGTAGAACTGGATGGCGACGTTAATGGTCACAAGTTCAGCGTTAGTGGAGAGGGTGAAGGTGATGCGACCTATGGCAAACTGACCCTGAAGCTGATCTGCACAACCGGCAAGCTGCCTGTTCCTTGGCCGACACTGGTTACAACGCTGGGCTATGGCGTACAATGTTTCGCACGGTACCCGGACCACATGAAGCAACATGACTTCTTCAAGAGCGCTATGCCTGAAGGCTATGTCCAAGAAAGGACTATCTTCTTCAAAGACGACGGCAATTACAAGACACGGGCCGAAGTCAAATTCGAAGGCGATACGCTGGTCAACAGAATCGAGCTGAAAGGCATCGACTTCAAGGAAGATGGCAACATCCTGGGCCATAAACTGGAATATAATTATAACAGTCATAATGTGTATATCACCGCTGACAAACAAAAGAATGGCATCAAGGCCAACTTCAAAATCAGACATAACATCGAAGATGGAGGTGTTCAACTGGCAGACCACTACCAACAAAATACTCCGATCGGCGATGGCCCGGTGCTGCTGCCGGATAACCATTATCTGAGTTATCAAAGTAAGCTGAGCAAGGATCCGAACGAAAAAAGAGATCATATGGTTCTGCTGGAATTCGTAACGGCCGCGGGCATCACGCATGGCATGGACGAGCTGTATAAAAGTAAAGGAGAAGAACTTTTCACTGGAGTTGTCCCAATTCTTGTTGAATTAGATGGTGATGTTAATGGGCACAAATTTTCTGTCAGTGGAGAGGGTGAAGGTGATGCAACATACGGAAAACTTACCCTTAAATTTATTTGCACTACTGGAAAACTACCTGTTCCATGGCCAACACTTGTCACTACTCTCACTTGGGGTGTTCAATGCTTTGCAAGATACCCAGATCATATGAAACAGCATGACTTTTTCAAGAGTGCCATGCCCGAAGGTTATGTACAGGAAAGAACTATATTTTTCAAAGATGACGGGAACTACAAGACACGTGCTGAAGTCAAGTTTGAAGGTGATACCCTTGTTAATAGAATCGAGTTAAAAGGTATTGATTTTAAAGAAGATGGAAACATTCTTGGACACAAATTGGAATACAACTACATCTCAGACAATGTATACATCACGGCAGACAAACAAAAGAATGGAATCAAAGCTAACTTCAAAATTAGACACAACATTGAAGATGGAGGCGTTCAACTAGCAGACCATTATCAACAAAATACTCCAATTGGCGATGGCCCTGTCCTTTTACCAGACAACCATTACCTGTCCACACAATCTAAGCTTTCGAAAGATCCCAACGAAAAGAGAGACCACATGGTCCTTCTTGAGTTTGTAACAGCTGCTGGGATTACACATGGCATGGATGAACTATACAAAtaaATGTCCAGACCTGCAGGCATGCAAGCTCTAGAGGCATCAAATAAAACGAAAGGCTCAGTCGAAAGACTGGGCCTTTCGTTTTATCTGTTGAAATGCACCAAAAACTCGTAAAAGCTCTGATGTATCTATCTTTTTTACACCGTTTTCATCTGTGCATATGGACAGTTTTCCCTTTGATATCTAACGGTGAACAGTTGTTCTACTTTTGTTTGTTAGTCTTGATGCTTCACTGATAGATACAAGAGCCATAAGAACCTCAGATCCTTCCGTATTTAGCCAGTATGTTCTCTAGTGTGGTTCGTTGTTTTTGCGTGAGCCATGAGAACGAACCATTGAGATCATGCTTACTTTGCATGTCACTCAAAAATTTTGCCTCAAAACTGGTGAGCTGAATTTTTGCAGTTAAAGCATCGTGTAGTGTTTTTCTTAGTCCGTTACGTAGGTAGGAATCTGATGTAATGGTTGTTGGTATTTTGTCACCATTCATTTTTATCTGGTTGTTCTCAAGTTCGGTTACGAGATCCATTTGTCTATCTAGTTCAACTTGGAAAATCAACGTATCAGTCGGGCGGCCTCGCTTATCAACCACCAATTTCATATTGCTGTAAGTGTTTAAATCTTTACTTATTGGTTTCAAAACCCATTGGTTAAGCCTTTTAAACTCATGGTAGTTATTTTCAAGCATTAACATGAACTTAAATTCATCAAGGCTAATCTCTATATTTGCCTTGTGAGTTTTCTTTTGTGTTAGTTCTTTTAATAACCACTCATAAATCCTCATAGAGTATTTGTTTTCAAAAGACTTAACATGTTCCAGATTATATTTTATGAATTTTTTTAACTGGAAAAGATAAGGCAATATCTCTTCACTAAAAACTAATTCTAATTTTTCGCTTGAGAACTTGGCATAGTTTGTCCACTGGAAAATCTCAAAGCCTTTAACCAAAGGATTCCTGATTTCCACAGTTCTCGTCATCAGCTCTCTGGTTGCTTTAGCTAATACACCATAAGCATTTTCCCTACTGATGTTCATCATCTGAGCGTATTGGTTATAAGTGAACGATACCGTCCGTTCTTTCCTTGTAGGGTTTTCAATCGTGGGGTTGAGTAGTGCCACACAGCATAAAATTAGCTTGGTTTCATGCTCCGTTAAGTCATAGCGACTAATCGCTAGTTCATTTGCTTTGAAAACAACTAATTCAGACATACATCTCAATTGGTCTAGGTGATTTTAATCACTATACCAATTGAGATGGGCTAGTCAATGATAATTACTAGTCCTTTTCCTTTGAGTTGTGGGTATCTGTAAATTCTGCTAGACCTTTGCTGGAAAACTTGTAAATTCTGCTAGACCCTCTGTAAATTCCGCTAGACCTTTGTGTGTTTTTTTTGTTTATATTCAAGTGGTTATAATTTATAGAATAAAGAAAGAATAAAAAAAGATAAAAAGAATAGATCCCAGCCCTGTGTATAACTCACTACTTTAGTCAGTTCCGCAGTATTACAAAAGGATGTCGCAAACGCTGTTTGCTCCTCTACAAAACAGACCTTAAAACCCTAAAGGCTTAAGTAGCACCCTCGCAAGCTCGGGCAAATCGCTGAATATTCCTTTTGTCTCCGACCATCAGGCACCTGAGTCGCTGTCTTTTTCGTGACATTCAGTTCGCTGCGCTCACGGCTCTGGCAGTGAATGGGGGTAAATGGCACTACAGGCGCCTTTTATGGATTCATGCAAGGAAACTACCCATAATACAAGAAAAGCCCGTCACGGGCTTCTCAGGGCGTTTTATGGCGGGTCTGCTATGTGGTGCTATCTGACTTTTTGCTGTTCAGCAGTTCCTGCCCTCTGATTTTCCAGTCTGACCACTTCGGATTATCCCGTGACAGGTCATTCAGACTGGCTAATGCACCCAGTAAGGCAGCGGTATCATCAACAGGCTTACCCGTCTTACTGTCAACCAGACCCGCCAGGATTCTCATGCATAAAAAACGCCCGGCGGCAACCGAGCGTTCTGAATTAATTAATCATCGCGACTGCAGAGATCCCAGAGTCCTTATTAGAAGAACTCGTCAAGAAGGCGATAGAAGGCGATGCGCTGCGAATCGGGAGCGGCGATACCGTAAAGCACGAGGAAGCGGTCAGCCCATTCGCCGCCAAGCTCTTCAGCAATATCACGGGTAGCCAACGCTATGTCCTGATAGCGGTCCGCCACACCCAGCCGGCCACAGTCGATGAATCCAGAAAAGCGGCCATTTTCCACCATGATATTCGGCAAGCAGGCATCGCCATGGGTCACGACGAGATCCTCGCCGTCGGGCATGCGCGCCTTGAGCCTG"

# Align

In [ ]:
def read_ab1(ab1_file, array_wrapper=np.array):
    seq = SeqIO.read(ab1_file, "abi")
    quality = seq.letter_annotations["phred_quality"]
    peak_calls = seq.annotations["abif_raw"]["PLOC2"]
    order = seq.annotations["abif_raw"]["FWO_1"]
    sequence = str(seq.seq)
    # Extract color data
    peaks = {}
    for i, (base, r) in enumerate(zip(order, [9, 10, 11, 12])):
        peaks[base] = array_wrapper(seq.annotations["abif_raw"]["DATA{}".format(r)])
    return {
        "name": seq.id,
        "sequence": sequence,
        "peaks": peaks,
        "peak_calls": array_wrapper(peak_calls),
        "quality": array_wrapper(quality),
    }

In [ ]:
def write_fasta(seqs, handle):
    # records = [SeqRecord(Seq(seq, IUPAC.DNA), name) for name, seq in seqs.items()]
    records = [SeqRecord(Seq(seq), name) for name, seq in seqs.items()]
    SeqIO.write(records, handle, "fasta")

In [ ]:
def call_subprocess(args, input_data=None):
    if input_data is not None:
        stdin = subprocess.PIPE
    else:
        stdin = None
    proc = subprocess.Popen(
        args, stdin=stdin, stdout=subprocess.PIPE, stderr=subprocess.PIPE
    )
    if input_data is not None:
        input_bytes = input_data.encode("utf-8")
    else:
        input_bytes = None
    stdout, stderr = proc.communicate(input_bytes)
    if stderr:
        raise Exception('got stderr: "{}"'.format(stderr))
    return stdout


#     if stdin is not None:
#         proc.stdin.write(input_data.encode('utf-8'))
#         proc.stdin.close()
#     while proc.returncode is None:
#         proc.poll()
#     return proc.stdout.read()

In [ ]:
def align_clustalomega(seqs):
    input_data = StringIO()
    write_fasta(seqs, input_data)
    output_data = call_subprocess(
        ["clustalo", "--infile=-", "--seqtype=DNA", "--outfmt=clustal"],
        input_data.getvalue(),
    )
    # return list(SeqIO.parse(StringIO(output_data.decode('utf-8')), 'clustal'))#, IUPAC.DNA)
    return next(
        AlignIO.parse(StringIO(output_data.decode("utf-8")), "clustal")
    )  # , IUPAC.DNA)

In [ ]:
def align_mafft(seqs):
    with tempfile.NamedTemporaryFile() as input_handle:
        write_fasta(seqs, input_handle.name)  # TODO: write directly using file handle
        output_data = call_subprocess(
            ["mafft", "--quiet", "--adjustdirection", input_handle.name]
        )
        return next(
            AlignIO.parse(StringIO(output_data.decode("utf-8")), "fasta")
        )  # , IUPAC.DNA)

In [ ]:
ab1_files = [
    "testseq/242a7-ITS4.ab1",
    "testseq/243a7-ITS4.ab1",
    "testseq/252a7-ITS4.ab1",
    "testseq/253a7-ITS4.ab1",
]
ab1s = {os.path.basename(ab1_file): read_ab1(ab1_file) for ab1_file in ab1_files}
seqs = {name: ab1["sequence"] for name, ab1 in ab1s.items()}
seqs = {"ref": ref_seq, **seqs}

In [ ]:
msa1 = align_clustalomega(seqs)

In [ ]:
msa2 = align_mafft(seqs)

## Trim alignments

In [ ]:
def trim_to_ref(msa, ref_name="ref"):
    ary = np.array([list(str(s.seq)) for s in msa1 if s.description != ref_name])
    mask = (ary == "-").sum(axis=0) >= len(msa) - 1
    idx_start = (~mask).argmax()
    idx_stop = (~mask)[::-1].argmax()
    return msa[:, idx_start:idx_stop]

In [ ]:
trim_to_ref(msa2)

# Chromatogram

In [ ]:
def show_chromatogram_alignment(
    ab1_files,
    ref_seq,
    aligner=align_mafft,
    margin=10,
    detail_width=100,
    plot_width=900,
    base_width=10,
    detail_base_width=10,
):
    ab1s = [read_ab1(ab1_file) for ab1_file in ab1_files]
    # sequences = [ab1['sequence']+'-'*max(max_length - len(ab1['sequence']), 0) for ab1 in ab1s]
    sequences = {"ref": ref_seq, **{ab1["name"]: ab1["sequence"] for ab1 in ab1s}}
    msa_untrimmed = trim_to_ref(aligner(sequences))
    # msa = trim_to_ref(msa_untrimmed)
    msa = msa_untrimmed
    # calculate dimensions
    sequence_strs = [str(m.seq) for m in msa]
    max_length = max(len(seq) for seq in sequence_strs)
    bases_per_row = plot_width // base_width
    grouped_sequences = [list(grouper(seq, bases_per_row)) for seq in sequence_strs]
    num_rows = max(len(row) for row in grouped_sequences)
    # print(num_rows,'-',len(grouped_sequences),'--',num_seqs)
    # num_rows = int(np.ceil(max_length / bases_per_row))
    num_seqs = len(sequence_strs)
    # make figures
    overview_plots = []
    overview_boxes = []
    overview_x_range = Range1d(
        -base_width, bases_per_row + base_width
    )  # is margin right?
    overview_y_range = Range1d(17 * (num_seqs - 1), -17)
    for row in range(num_rows):
        p = figure(
            plot_width=plot_width,
            plot_height=12 * num_seqs,
            tools="tap,save",
            x_range=overview_x_range,
            y_range=overview_y_range,
            output_backend="webgl",
        )
        p.axis.visible = False
        p.min_border = 0
        p.min_border_bottom = 0
        p.grid.visible = False
        p.outline_line_color = None
        overview_plots.append(p)
        for seq in range(num_seqs):
            sequence = grouped_sequences[seq][row]
            p.text(
                x=range(len(sequence)),
                y=17 * seq,
                text=[s for s in sequence],
                text_align="center",
                text_baseline="bottom",
                text_font="Courier",
                text_font_size="12px",
            )
        box = BoxAnnotation(left=10, right=40, fill_color="blue", fill_alpha=0.5)
        p.add_layout(box)
        overview_boxes.append(box)
        # overview_plots[-1].min_border_bottom = 5
    # overview_plots[-1].min_border_bottom = 30
    # overview_plots[-1].height += 30
    detail_plots = []
    detail_x_range = None
    for aligned_sequence, ab1 in zip(sequence_strs[1:], ab1s):
        name, sequence, peaks, peak_calls, quality = map(
            ab1.__getitem__, ("name", "sequence", "peaks", "peak_calls", "quality")
        )
        if detail_x_range is None:
            x_min = np.min(peak_calls)
            x_max = np.max(peak_calls)
            detail_x_range = Range1d(
                x_min,
                x_min + detail_width,
                bounds=(x_min - margin, x_max + margin),
                max_interval=detail_width,
            )
        p = figure(
            plot_width=plot_width,
            plot_height=50,
            output_backend="webgl",
            # tools=['xpan', 'reset','xwheel_zoom', BoxZoomTool(dimensions='width')],
            tools=["xpan"],
            active_drag="xpan",
            # active_scroll='xwheel_zoom',
            # title=name,
            x_range=detail_x_range,
            # y_range=y_range
        )
        p.extra_y_ranges = {"q": Range1d(start=0, end=100)}
        p.add_layout(LinearAxis(y_range_name="q"), "right")
        local_start = 0
        for match in re.finditer(r"([^-]+)", aligned_sequence):
            aligned_start = match.start()
            aligned_end = match.end()
            sequence_section = aligned_sequence[aligned_start:aligned_end]
            section_length = len(sequence_section)
            local_end = local_start + section_length - 1
            quality_section = quality[local_start : local_end + 1]
            # print('qs',len(quality_section),'adist',aligned_start,aligned_end-1,'diff',aligned_end-1-aligned_start)
            quality_x = np.linspace(
                aligned_start, aligned_end, len(quality_section), endpoint=False
            )
            p.rect(
                x=quality_x,
                y=quality_section / 2,
                width=1,
                height=quality_section,
                y_range_name="q",
                color="#e7ecf6",
            )
            for color, base in zip(
                ["red", "green", "blue", "black"], ["T", "A", "C", "G"]
            ):
                peak = peaks[base]
                # peak_section = peak[peak_calls[local_start]:peak_calls[local_end]]
                # p.line(np.linspace(aligned_start, aligned_end-1, len(peak_section)), peak_section, line_color=color)
                peak_left_start = int(
                    (peak_calls[local_start - 1] + peak_calls[local_start]) / 2
                )
                peak_left = peak[peak_left_start : peak_calls[local_start]]
                peak_left_x = np.linspace(
                    aligned_start - 0.5, aligned_start, len(peak_left), endpoint=False
                )
                peak_section = peak[peak_calls[local_start] : peak_calls[local_end]]
                peak_section_x = np.linspace(
                    aligned_start, aligned_end - 1, len(peak_section), endpoint=False
                )
                if local_end + 1 < len(peak_calls):
                    peak_right_end = int(
                        (peak_calls[local_end] + peak_calls[local_end + 1]) / 2
                    )
                else:
                    peak_right_end = peak_calls[local_end]
                peak_right = peak[peak_calls[local_end] : peak_right_end]
                peak_right_x = np.linspace(
                    aligned_end - 1, aligned_end - 0.5, len(peak_right), endpoint=False
                )
                # p.line(peak_section_x, peak_section, line_color=color)
                peak_x = np.concatenate((peak_left_x, peak_section_x, peak_right_x))
                peak_y = np.concatenate((peak_left, peak_section, peak_right))
                p.line(peak_x, peak_y, line_color=color)
            local_start += len(sequence_section)
        detail_plots.append(p)
        p2 = figure(
            plot_width=plot_width,
            plot_height=30,
            tools=["xpan"],
            active_drag="xpan",
            x_range=detail_x_range,
            output_backend="webgl",
        )
        r = p2.text(
            x=range(len(aligned_sequence)),
            y=0.1,
            text=[s for s in aligned_sequence],
            text_align="center",
            text_baseline="middle",
        )
        # r.glyph.text_font_style="bold"
        detail_plots.append(p2)
        p.min_border = 0
        p2.min_border = 0
        p.grid.visible = False
        p2.grid.visible = False
        p.outline_line_color = None
        p2.outline_line_color = None
    for p in detail_plots:
        p.xaxis.visible = False
    for p in detail_plots:
        p.yaxis.visible = False
    detail_plots[-1].height = 50
    detail_plots[-1].xaxis.visible = True
    # callback
    BOX_JS = """
    for (i=0; i<boxes.length; i++) {
        var start_in_row = false;
        var end_in_row = false;
        if ((i*bases_per_row <= start) && (start < (i+1)*bases_per_row)) {
           start_in_row = true;
        }
        if ((i*bases_per_row <= end) && (end < (i+1)*bases_per_row)) {
           end_in_row = true;
        }
        //console.log('row',i,'startinrow',start_in_row,'endinrow',end_in_row);
        if (start_in_row || end_in_row) {
           boxes[i].visible = true;
        }
        else {
           boxes[i].visible = false;
        }
        if (start_in_row) {
           boxes[i].left = (start-i*bases_per_row);
           boxes[i].right = bases_per_row;
        }
        else {
           boxes[i].left = 0;
        }
        if (end_in_row) {
           boxes[i].left = 0;
           boxes[i].right = (end-i*bases_per_row);
        }
        else {
           boxes[i].right = bases_per_row;
        }
        if ((i*bases_per_row > start) && (end >= (i+1)*bases_per_row)) {
           boxes[i].visible = true;
           boxes[i].left = 0;
           boxes[i].right = bases_per_row;
        }
    }
    """
    detail_scroll_callback = CustomJS(
        args=dict(boxes=overview_boxes, x_range=detail_plots[0].x_range),
        code="""
                                           var bases_per_row = {bases_per_row};
                                           var start = x_range.start;
                                           var end = x_range.end;
                                           """.format(
            bases_per_row=bases_per_row
        )
        + BOX_JS,
    )
    detail_plots[0].x_range.js_on_change("end", detail_scroll_callback)
    for row in range(num_rows):
        range_callback = CustomJS(
            args=dict(boxes=overview_boxes, plot=detail_plots[0]),
            code="""
                                       var row = {row};
                                       var bases_per_row = {bases_per_row};
                                       var detail_width = {detail_width};
                                       var length = {length};
                                       var start = Math.max(cb_obj.x + row*bases_per_row - Math.floor(detail_width / 2), 0);
                                       var end = start + detail_width; // also bound
                                       if (end > length) {{
                                           end = length;
                                           start = end - detail_width;
                                       }}
                                       //plot.trigger('change');
                                       plot.x_range.start = start;
                                       plot.x_range.end = end;
                                       """.format(
                row=row,
                bases_per_row=bases_per_row,
                detail_width=detail_width,
                length=max_length,
            ),
        )
        overview_plots[row].js_on_event("tap", range_callback)
    show(
        column(
            gridplot([[p] for p in overview_plots]),
            gridplot([[p] for p in detail_plots]),
        )
    )


# show_chromatogram_alignment(['testseq/242a7-ITS4.ab1', 'testseq/243a7-ITS4.ab1', 'testseq/252a7-ITS4.ab1', 'testseq/253a7-ITS4.ab1'], ref_seq)
show_chromatogram_alignment(
    ["testseq/242a7-ITS4.ab1", "testseq/243a7-ITS4.ab1", "testseq/252a7-ITS4.ab1"],
    ref_seq,
)

In [ ]:
# red highlights for disagreements
# reference sequence in bold on top of detail plot
# change box color
# fix boxes on startup
# figure out overview margins
# export to static html
# debug load time
# TODO: plot overview/detail EACH AS ONE PLOT (stacking in y-axis)
# TODO: unrolled/wrapped chromatograph viewer (reflowing on window resize)